In [1]:
# importing all the required libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import Select

import xlsxwriter
import time

In [2]:
# adding options

chromeOptions = Options()
chromeOptions.add_argument('--kiosk')
chromeOptions.add_argument('blink-settings=imagesEnabled=false')
chromeOptions.page_load_strategy = 'normal'

In [13]:
# opening the webpage

driver = webdriver.Chrome('../chromedriver/chromedriver', options=chromeOptions)

In [4]:
driver.get('https://www.producthunt.com')

In [10]:
driver.close()

# Extract Product Hunt listings
### Steps:
1. Scroll down the page
2. Click all the show more button (to be able to scrape all data)
3. Scrape the dates (for reference: to see which timeframe was scraped)
4. Get all the cards
5. Loop the cards then scrape all the necessary data(then add it into a list):
    - Product Name
    - Description
    - Tags
    - Votes
    - href (to get the URL since it doesn't exist in the home page)  
6. Loop the list and go to each href then scrape the website to get the URL.
7. Save the lists with complete information to an excel file.

# Step 1: Scroll down

In [5]:
for i in range(300): # scrolling down the page
    print(i, driver.execute_script("return document.body.scrollHeight"))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1) # give the browser some time to load

0 3528
1 3921
2 3921
3 6137
4 7465
5 8793
6 10121
7 10514
8 11449
9 11862
10 12797
11 13190
12 14125
13 15453
14 16781
15 18109
16 19457
17 20785
18 22133
19 23481
20 24809
21 26137
22 27465
23 28793
24 30121
25 31449
26 32777
27 34105
28 35433
29 36761
30 38089
31 39417
32 40745
33 41138
34 42093
35 43421
36 44769
37 46097
38 47425
39 48753
40 50101
41 51429
42 52757
43 54085
44 55413
45 56741
46 58069
47 59397
48 60725
49 62073
50 63401
51 64729
52 66057
53 67385
54 68713
55 70041
56 71413
57 72741
58 74069
59 75397
60 76725
61 78073
62 79401
63 80729
64 82057
65 83385
66 84713
67 86041
68 87369
69 88697
70 90025
71 91353
72 92681
73 94009
74 95357
75 96685
76 98013
77 99341
78 100669
79 101997
80 103325
81 104653
82 105981
83 107329
84 108657
85 109985
86 111313
87 112641
88 113969
89 115317
90 116645
91 117973
92 119301
93 120629
94 121957
95 123285
96 124633
97 125961
98 127289
99 128617
100 129010
101 129945
102 131273
103 132621
104 133949
105 135297
106 136625
107 137953
108 13

In [30]:
print(driver.current_url)

https://www.producthunt.com/


# Step 2: Click all show more buttons

In [8]:
# click all show more buttons
show_more = driver.find_elements_by_class_name('showHiddenPosts_fbc1f')
print(len(show_more))
for show in show_more:
    show.click()

192


# Step 3: Take all dates and print it

In [7]:
# get all dates (for reference as to when to when we got the dates.)
get_all_date = driver.find_element_by_tag_name('main').find_elements_by_class_name('header_fb2c3')
print(len(get_all_date))
for date in get_all_date: 
    print(date.text)

296
Upcoming
powered by Ship
Today
POPULARNEWEST
Best products last week
Yesterday
April 18th
April 17th
April 16th
April 15th
April 14th
April 13th
April 12th
April 11th
April 10th
April 9th
April 8th
April 7th
April 6th
April 5th
April 4th
April 3rd
April 2nd
April 1st
March 31st
March 30th
March 29th
March 28th
March 27th
March 26th
March 25th
March 24th
March 23rd
March 22nd
March 21st
March 20th
March 19th
March 18th
March 17th
March 16th
March 15th
March 14th
March 13th
March 12th
March 11th
March 10th
March 9th
March 8th
March 7th
March 6th
March 5th
March 4th
March 3rd
March 2nd
March 1st
February 29th
February 28th
February 27th
February 26th
February 25th
February 24th
February 23rd
February 22nd
February 21st
February 20th
February 19th
February 18th
February 17th
February 16th
February 15th
February 14th
February 13th
February 12th
February 11th
February 10th
February 9th
February 8th
February 7th
February 6th
February 5th
February 4th
February 3rd
February 2nd
February 1st

# Step 4: Find all cards

In [9]:
cards = driver.find_elements_by_class_name('item_54fdd') # getting all cards from classname
print(len(cards))
products = []

6471


# Step 5: Loop cards and extract data

In [10]:
for i, card in enumerate(cards):
    try:
        title = card.find_element_by_tag_name('h3').text # getting title from h3 in each li
        description = card.find_element_by_tag_name('p').text # getting description from p in each li
        vote = card.find_element_by_class_name('voteButtonWrap_4c515').find_element_by_tag_name('span').text # getting vote from classname then span
        href = card.find_element_by_tag_name('a').get_attribute('href') # get href through anchor tag
        products.append({'product': title, 'description': description, 'tag': [], 'vote': vote, 'url': None, 'href': href})
    except:
        print('error', i)

error 6311


In [11]:
print(len(products))
# for i in products:
#     print(i)

6470


# Step 6: Go to each website and get the URL and tags

In [31]:
for product in products:
    if(product['url'] == None):
        driver.get(product['href'])
        try:
            driver.find_element_by_class_name('more_2e78b').click()
        except:
            print('No clickable tag')
        for tag in driver.find_elements_by_class_name('button_53e93'):
            product['tag'].append(tag.find_element_by_tag_name('a').get_attribute('title'))
        driver.get(driver.find_element_by_class_name('link_9bebc').get_attribute('href'))
        product['url'] = driver.current_url
        print(product)

{'product': 'Coronavirus checklist', 'description': 'A checklist to help startups cope with the coronavirus', 'tag': ['Health and Fitness', 'SaaS'], 'vote': '119', 'url': 'https://www.coronaviruschecklist.co/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/coronavirus-checklist'}
No clickable tag
{'product': 'Take the Timer', 'description': 'Simple interval training timer', 'tag': ['Health and Fitness'], 'vote': '93', 'url': 'https://www.takethetimer.com/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/take-the-timer'}
{'product': 'REALITYLINE', 'description': "World's First Online Startup Launcher.", 'tag': ['Developer Tools', 'Artificial Intelligence', 'Tech', 'Side Projects', 'Productivity', 'Venture Capital'], 'vote': '76', 'url': 'https://realityline.ca/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/realityline'}
{'product': 'Aiava', 'description': 'Customer journey mapping for the real world', 'tag': ['Customer communication', 'Design Tool

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".link_9bebc"}
  (Session info: chrome=81.0.4044.113)


In [42]:
count = 0
for (i, product) in enumerate(products):
    if(product['url'] == None):
        print(product)
        count += 1
#         del products[i]
#         break
print(count)

{'product': 'Vegan Translations', 'description': 'Useful phrases for vegans in other languages', 'tag': [], 'vote': '20', 'url': None, 'href': 'https://www.producthunt.com/posts/vegan-translations'}
{'product': 'MsgMentor', 'description': 'Coaches your team to use Slack more efficiently 💰', 'tag': [], 'vote': '18', 'url': None, 'href': 'https://www.producthunt.com/posts/msgmentor'}
{'product': 'Notatio', 'description': 'Generate wacky story ideas in a MadLib-like fashion', 'tag': [], 'vote': '21', 'url': None, 'href': 'https://www.producthunt.com/posts/notatio'}
{'product': 'Read This Twice', 'description': 'Verified book recommendations from people we look up to', 'tag': [], 'vote': '1008', 'url': None, 'href': 'https://www.producthunt.com/posts/read-this-twice'}
{'product': 'Covy', 'description': 'Visualize the growth of COVID-19 on our map (US based)', 'tag': [], 'vote': '395', 'url': None, 'href': 'https://www.producthunt.com/posts/covy'}
{'product': 'Figma Tracking', 'description'

{'product': 'Outsell.ai', 'description': 'Free email verification for Salesforce', 'tag': [], 'vote': '48', 'url': None, 'href': 'https://www.producthunt.com/posts/outsell-ai'}
{'product': 'Village', 'description': 'See discussions, questions and conversations around you', 'tag': [], 'vote': '42', 'url': None, 'href': 'https://www.producthunt.com/posts/village-3'}
{'product': 'Hard G', 'description': 'Find and save GIFs on your Mac', 'tag': [], 'vote': '37', 'url': None, 'href': 'https://www.producthunt.com/posts/hard-g'}
{'product': 'Reddex', 'description': 'A toolkit for YouTube Reddit narrators', 'tag': [], 'vote': '32', 'url': None, 'href': 'https://www.producthunt.com/posts/reddex'}
{'product': 'Undiet Tracker', 'description': 'Track your weight loss progress', 'tag': [], 'vote': '30', 'url': None, 'href': 'https://www.producthunt.com/posts/undiet-tracker'}
{'product': 'MicroAcquire', 'description': 'A free & anonymous startup acquisition marketplace', 'tag': [], 'vote': '1120', '

{'product': 'Triode', 'description': 'Enjoy your favorite Internet radio stations wherever you go.', 'tag': [], 'vote': '79', 'url': None, 'href': 'https://www.producthunt.com/posts/triode'}
{'product': 'Jahwow', 'description': "Easiest way to plan, organise and manage your child's party", 'tag': [], 'vote': '33', 'url': None, 'href': 'https://www.producthunt.com/posts/jahwow'}
{'product': 'Nabsack', 'description': 'Buy and sell app for locals, made simple.', 'tag': [], 'vote': '29', 'url': None, 'href': 'https://www.producthunt.com/posts/nabsack'}
{'product': 'Blaze', 'description': 'Fast peer to peer file sharing web app ⚡', 'tag': [], 'vote': '584', 'url': None, 'href': 'https://www.producthunt.com/posts/blaze-2'}
{'product': 'Flow Lines', 'description': 'Mathematical art SVG creator with infinite variations', 'tag': [], 'vote': '456', 'url': None, 'href': 'https://www.producthunt.com/posts/flow-lines'}
{'product': 'Apple Music Replay', 'description': "Apple Music's answer to Spotif

{'product': 'Reis & Irvys', 'description': 'Fro-yo robot', 'tag': [], 'vote': '33', 'url': None, 'href': 'https://www.producthunt.com/posts/reis-irvys'}
{'product': 'Canon EOS 90d', 'description': 'A new full fledged DSLR for enthusiasts', 'tag': [], 'vote': '27', 'url': None, 'href': 'https://www.producthunt.com/posts/canon-eos-90d'}
{'product': 'litty', 'description': 'Find lit clubs & bars near you', 'tag': [], 'vote': '32', 'url': None, 'href': 'https://www.producthunt.com/posts/litty'}
{'product': 'SVG Artista', 'description': 'Little tool that helps you create SVG animations', 'tag': [], 'vote': '917', 'url': None, 'href': 'https://www.producthunt.com/posts/svg-artista'}
{'product': 'Glorify', 'description': 'Create beautiful product images in a few clicks', 'tag': [], 'vote': '1107', 'url': None, 'href': 'https://www.producthunt.com/posts/glorify-2'}
{'product': 'Treefort', 'description': 'A simple way to involve your users directly in your roadmap', 'tag': [], 'vote': '414', 'u

# Fixing Datas

In [44]:
# url == href
for product in products:
    if(product['url'] == product['href']):
        driver.get(product['href'])
        driver.get(driver.find_element_by_class_name('link_9bebc').get_attribute('href'))
        product['url'] = driver.current_url
        print(product)

{'product': 'Snapup', 'description': 'Capture anything you want on the web, incredibly quickly', 'tag': [], 'vote': '64', 'url': 'https://snapup.xyz/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/snapup-3'}
{'product': 'How to launch a delivery business', 'description': 'A guide to pivot your business in a matter of days.', 'tag': [], 'vote': '42', 'url': 'https://blog.routific.com/how-to-launch-and-scale-a-delivery-business-profitably?ref=producthunt', 'href': 'https://www.producthunt.com/posts/how-to-launch-a-delivery-business'}
{'product': 'Neptune', 'description': 'Twitter done right, just for you.', 'tag': [], 'vote': '38', 'url': 'https://www.neptuneapp.net/?ref=producthunt', 'href': 'https://www.producthunt.com/posts/neptune'}


KeyboardInterrupt: 

In [46]:
# products with trailing tags
for product in products:
    correctTags = True
    for tag in product['tag']:
        if('...' in tag):
            correctTags = False
            print(tag)
    if(not correctTags):
        driver.get(product['href'])
        product['tag'] = []
        try:
            driver.find_element_by_class_name('more_2e78b').click()
        except:
            print('No clickable tag')
        for tag in driver.find_elements_by_class_name('button_53e93'):
            product['tag'].append(tag.find_element_by_tag_name('a').get_attribute('title'))
        
        print(product)

In [23]:
print(len(products))

6470


# Step 7: Put data into excel file

In [12]:
workbook = xlsxwriter.Workbook('producthunt.xlsx')
worksheet = workbook.add_worksheet()
worksheet.set_column('A:D', 25)
worksheet.set_column('E:E', 70)
worksheet.write(0, 0, 'Product Name')
worksheet.write(0, 1, 'Description')
worksheet.write(0, 2, 'URL')
worksheet.write(0, 3, 'ProductHunt URL')
worksheet.write(0, 4, 'Tag')
worksheet.write(0, 5, 'Votes')

row = 1
for product in products:
    worksheet.write(row, 0, product['product'])
    worksheet.write(row, 1, product['description'])
    worksheet.write(row, 2, product['url'].split('?')[0])
    worksheet.write(row, 3, product['href'])
    if(product['tag'][0] == 'nan'):
        worksheet.write(row, 4, ','.join(product['tag'][1:]))
    else:
        worksheet.write(row, 4, ','.join(product['tag']))
    worksheet.write(row, 5, int(product['vote']))
    row += 1
    
workbook.close()

# Continuation
Import from excel and continue getting the urls

In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_excel('producthunt.xlsx')
products = []
print(df)

                     Product Name  \
0                      Good Books   
1                     SpatialChat   
2                           Snack   
3                           8base   
4     Payment Pages by involve.me   
...                           ...   
6464             The New Girlboss   
6465                      ProGram   
6466                    Adv3nture   
6467                 Squirrly SPY   
6468                     GANpaint   

                                            Description  \
0     Largest curated collection of 6,500+ book reco...   
1                Your virtual party for up to 50 guests   
2     Have spontaneous, time-restricted video conver...   
3     The GraphQL Backend that harnesses the full po...   
4     Collect Stripe & PayPal payments online with c...   
...                                                 ...   
6464    A new professional network for millennial women   
6465     CLI based, self-hosted Instagram-esque gallery   
6466                   3

In [7]:
for product in df.iloc:
    products.append({'product': str(product['Product Name']), 'description': str(product['Description']), 'tag': str(product['Tag']).split(','), 'vote': int(product['Votes']), 'url': str(product['URL']), 'href': str(product['ProductHunt URL'])})


In [8]:
print(len(products))
for product in products:
    print(product)

6469
{'product': 'Good Books', 'description': 'Largest curated collection of 6,500+ book recommendations.', 'tag': ['Productivity', 'History books'], 'vote': 685, 'url': 'https://www.goodbooks.io/', 'href': 'https://www.producthunt.com/posts/good-books'}
{'product': 'SpatialChat', 'description': 'Your virtual party for up to 50 guests', 'tag': ['User experience', 'Web app', 'Video streaming'], 'vote': 400, 'url': 'https://spatial.chat/', 'href': 'https://www.producthunt.com/posts/spatialchat'}
{'product': 'Snack', 'description': 'Have spontaneous, time-restricted video conversations.', 'tag': ['Slack', 'Bots'], 'vote': 274, 'url': 'https://aboutsnack.com/', 'href': 'https://www.producthunt.com/posts/snack-4'}
{'product': '8base', 'description': 'The GraphQL Backend that harnesses the full power of AWS.', 'tag': ['nan'], 'vote': 669, 'url': 'https://www.8base.com/producthunt', 'href': 'https://www.producthunt.com/r/p/171794?ref=home&promo_id=10852'}
{'product': 'Payment Pages by involve

{'product': 'Cypherace', 'description': 'Cypherace is web-based debugging tool & bug fixing engine', 'tag': ['Developer Tools', 'Web App', 'Productivity'], 'vote': 22, 'url': 'https://cypherace.com/', 'href': 'https://www.producthunt.com/posts/cypherace'}
{'product': 'Stonly Knowledge Base', 'description': 'Interactive knowledge bases and help-centers', 'tag': ['Tech', 'Web App', 'Customer communication'], 'vote': 676, 'url': 'https://stonly.com/product/knowledge-base-software', 'href': 'https://www.producthunt.com/posts/stonly-knowledge-base'}
{'product': 'Relephant', 'description': 'All of your screenshots, intelligently organized', 'tag': ['SaaS', 'Tech', 'Mac', 'Productivity'], 'vote': 665, 'url': 'https://try.relephant.in/', 'href': 'https://www.producthunt.com/posts/relephant'}
{'product': 'Stickies Capture by Miro', 'description': 'Digitize and edit handwritten sticky notes', 'tag': ['Apple', 'Web App', 'Productivity'], 'vote': 703, 'url': 'https://miro.com/stickies-capture/', '

{'product': 'Freshsales Mobile CRM', 'description': 'Your full-fledged CRM, to go', 'tag': ['Productivity', 'Sales', 'Marketing', 'SaaS', 'Tech', 'Android', 'iPhone'], 'vote': 165, 'url': 'https://www.freshworks.com/freshsales-crm/mobile/', 'href': 'https://www.producthunt.com/posts/freshsales-mobile-crm'}
{'product': 'Snapchat Ads Automation by Revealbot', 'description': 'Save time and automate your best Snapchat Ads strategies', 'tag': ['Advertising', 'Snapchat', 'Social Media Tools', 'Marketing'], 'vote': 126, 'url': 'https://revealbot.com/snapchat-ads-automation', 'href': 'https://www.producthunt.com/posts/snapchat-ads-automation-by-revealbot'}
{'product': 'Tech Top 10 by The Information', 'description': 'Today’s top 10 tech stories, analyzed for you', 'tag': ['News', 'Tech', 'Android', 'iPhone'], 'vote': 127, 'url': 'https://www.theinformation.com/top10', 'href': 'https://www.producthunt.com/posts/tech-top-10-by-the-information'}
{'product': 'SuperCook App', 'description': 'Find a

{'product': 'Permanent Record', 'description': "Edward Snowden's explosive memoir is now available", 'tag': ['nan'], 'vote': 298, 'url': 'nan', 'href': 'https://www.producthunt.com/posts/permanent-record'}
{'product': 'Bloom', 'description': 'Video fitness app for emotional health', 'tag': ['nan'], 'vote': 399, 'url': 'nan', 'href': 'https://www.producthunt.com/posts/bloom-d1827c3c-56f3-4b10-b1ee-802905fcbce1'}
{'product': 'Signum 2.0', 'description': 'Be in the know on emerging new trends - powered by AI', 'tag': ['nan'], 'vote': 232, 'url': 'nan', 'href': 'https://www.producthunt.com/posts/signum-2-0'}
{'product': 'Breaker for Android', 'description': 'Social podcasts are now available on Android', 'tag': ['nan'], 'vote': 196, 'url': 'nan', 'href': 'https://www.producthunt.com/posts/breaker-for-android'}
{'product': 'Byte Vitae', 'description': 'Generate great looking resumes for developers', 'tag': ['nan'], 'vote': 189, 'url': 'nan', 'href': 'https://www.producthunt.com/posts/byte-v

{'product': 'Moon FM for Mac', 'description': 'An easy to use podcast player for podcast lovers', 'tag': ['nan'], 'vote': 288, 'url': 'nan', 'href': 'https://www.producthunt.com/posts/moon-fm-for-mac'}
{'product': 'TypeFuse', 'description': 'A publishing platform where you can read & share articles', 'tag': ['nan'], 'vote': 218, 'url': 'nan', 'href': 'https://www.producthunt.com/posts/typefuse'}
{'product': 'Design Tools Weekly', 'description': 'Design tools & guides on using them, sent to your 📨', 'tag': ['nan'], 'vote': 312, 'url': 'nan', 'href': 'https://www.producthunt.com/posts/design-tools-weekly'}
{'product': 'Pipe.blue', 'description': 'The free link shortener for developers', 'tag': ['nan'], 'vote': 111, 'url': 'nan', 'href': 'https://www.producthunt.com/posts/pipe-blue'}
{'product': 'Maker Goals', 'description': 'A new way to manage your Product Hunt goals', 'tag': ['nan'], 'vote': 121, 'url': 'nan', 'href': 'https://www.producthunt.com/posts/maker-goals'}
{'product': 'Simple

In [14]:
# get product urls and tags
for product in products:
    if(product['url'] == 'nan'):
        driver.get(product['href'])
        try:
            driver.find_element_by_class_name('more_2e78b').click()
        except:
            print('No clickable tag')
        for tag in driver.find_elements_by_class_name('button_53e93'):
            product['tag'].append(tag.find_element_by_tag_name('a').get_attribute('title'))
        driver.get(driver.find_element_by_class_name('link_9bebc').get_attribute('href'))
        product['url'] = driver.current_url
        print(product)

KeyboardInterrupt: 